## Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [3]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [4]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)
test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [5]:
train['message'] = train['message'].str.lower()
test['message'] = test['message'].str.lower()

In [6]:
import string
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])

train['message'] = train['message'].apply(remove_punctuation_numbers)
test['message'] = test['message'].apply(remove_punctuation_numbers)

In [7]:
y = train['sentiment']
X = train['message']

In [8]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=.00001, random_state=20)

In [9]:
from sklearn.pipeline import Pipeline

In [10]:
text_clf = Pipeline([('tfidf',TfidfVectorizer(max_df=0.8,min_df=2,ngram_range=(1,2))),
                     ('clf',LinearSVC(C=0.3,class_weight='balanced',max_iter=3000))])

In [11]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('clf',
                 LinearSVC(C=0.3, class_weight='balanced', max_iter=3000))])

In [12]:
y_val_preds = text_clf.predict(X_val)

In [13]:
f1_score(y_val, y_val_preds, average="macro")

1.0

In [14]:
testx = test['message']

In [15]:
test_preds = text_clf.predict(testx)

In [16]:
test['sentiment'] = test_preds

In [17]:
test.head()

,message,tweetid,sentiment
0,europe will now be looking to china to make su...,169760,1
1,combine this with the polling of staffers re c...,35326,1
2,the scary unimpeachable evidence that climate ...,224985,1
3,karoli morgfair osborneink dailykos \nputin go...,476263,1
4,rt fakewillmoore female orgasms cause global w...,872928,0


## Creating an output csv for submission

In [18]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)